In [7]:
from get_embedding_function import get_embedding_function
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.llms.ollama import Ollama

DATA_PATH = "data"
CHROMA_PATH = "chroma"

PROMPT_TEMPLATE = """
    You are a medical assistant.
    Answer the question using ONLY the context below.
    If the answer is not contained in the context, say:
    "I dont know based on the provided documents."

    Context:    
    {context}

    ---

    Answer the question based on the above context: {question}
    """


In [8]:
def query_rag(query_text: str):
    embedding_function = get_embedding_function()
    db = Chroma(
        persist_directory=CHROMA_PATH,
        embedding_function=embedding_function
    )
    
    results = db.similarity_search_with_score(query_text, k=5)
    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)

    model = Ollama(model="mistral")
    response_text = model.invoke(prompt)
    print(response_text)
    sources = [doc.metadata.get("id", None) for doc, _score in results]
    formatted_response = f"Response: {response_text}\nSources: {sources}"
    print(formatted_response)
    return response_text

In [9]:
query_text = "What does the guideline say about treating malaria in pregnant women?"

query_rag(query_text)


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 988.65it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


 The guideline states that during pregnancy, treatment for uncomplicated falciparum malaria should be an artemisinin-based combination therapy (ACT) given by the oral route for 3 days. It does not mention any use of Primaquine in pregnant women, suggesting it is not recommended for them. For prevention, pregnant women in high-risk areas are referred to a guide called Essential obstetric and newborn care, MSF.
Response:  The guideline states that during pregnancy, treatment for uncomplicated falciparum malaria should be an artemisinin-based combination therapy (ACT) given by the oral route for 3 days. It does not mention any use of Primaquine in pregnant women, suggesting it is not recommended for them. For prevention, pregnant women in high-risk areas are referred to a guide called Essential obstetric and newborn care, MSF.
Sources: ['data/guideline-170-en.pdf:185:1', 'data/guideline-170-en.pdf:181:2', 'data/guideline-170-en.pdf:181:1', 'data/guideline-170-en.pdf:185:2', 'data/guidelin

' The guideline states that during pregnancy, treatment for uncomplicated falciparum malaria should be an artemisinin-based combination therapy (ACT) given by the oral route for 3 days. It does not mention any use of Primaquine in pregnant women, suggesting it is not recommended for them. For prevention, pregnant women in high-risk areas are referred to a guide called Essential obstetric and newborn care, MSF.'